#M.Lytova, M.Spanner, I.Tamblyn. Deep learning and high harmonic generation (2020)
##Codes for Section II.B : Regression model 

##Headers and constants

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers import Input, Dense, Dropout
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam, Nadam
from keras import objectives
from keras.losses import mean_squared_error
from keras.callbacks import TensorBoard
import argparse
import matplotlib.pyplot as plt
import time

In [ ]:
PI = 3.14159265359

x_n_points = 512   # number of nodes in input layer
x_n = np.linspace(0, 100, x_n_points) 

batch_size = 128

n_train = 500000   
n_test = 50000

##Training set generation

In [ ]:
# Training set
w0_train = np.random.rand(n_train,1)*0.5+0.5    # vector of random frequencies in [0.5, 1]
y_train = np.sin(w0_train*x_n)

##Testing set generation

In [ ]:
# Testing set
w0_test = np.random.rand(n_test,1)*0.5+0.5   # vector of random frequencies in [0.5, 1]
y_test = (np.sin(w0_test*x_n))

##Model

512$->$ 128 $->$ 64 $->$ 16 $->$ 1

In [ ]:
inputs = Input(shape = (x_n_points,))

x = Dense(128, activation='tanh')(inputs)
x = Dense(64, activation='tanh')(x)
x = Dense(16, activation='tanh')(x)

outputs = Dense(1, activation='tanh')(x)

ModelLam = Model(inputs, outputs)
opt = Nadam(lr=0.0001)
ModelLam.compile(optimizer=opt, loss='mean_squared_error') 

print(ModelLam.summary())

##Training

In [ ]:
tic = time.perf_counter()

history = ModelLam.fit(y_train, w0_train, 
                epochs=200,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(y_test, w0_test))

toc = time.perf_counter()
print(f"Execution time {toc - tic:0.4f} seconds")

def plot_losses():
    plt.figure(figsize=(8,4))
    plt.plot(np.log10(history.history['loss']),color='blue')
    plt.plot(np.log10(history.history['val_loss']),color='red')
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()

##Training and validation losses

In [ ]:
plot_losses()

##Prediction

In [ ]:
prediction = ModelLam.predict(y_test)

##Predicted vs True

In [ ]:
def plot_predict_true():
    fig = plt.figure(figsize=(4,4),constrained_layout=False)
    plt.scatter(w0_test[:,0], prediction[:,0], color="red", s = 0.5)
    plt.xlabel('True', fontsize=14)
    plt.ylabel('Predicted', fontsize=14)
    plt.grid()
    plt.show() 
    plt.close()  

plot_predict_true()      